In [1]:
import pandas as pd
import numpy as np
import scipy.stats as ss
import os
import re
import traceback
from happierfuntokenizing import Tokenizer
from __future__ import division

total_number_of_blog_posts = 0
total_number_of_users = 0
industry_users_map = {}
userid_industry_map = {}
userid_age_map = {}
userid_gender_map = {}

all_words_counts = {}


### 1) Method for reading all files, parsing the blogs and tokenizing them

In [2]:
def read_all_blogs(users_words_counts,users_topics_probs):
    
    global total_number_of_blog_posts
    global total_number_of_users
    global industry_users_map 
    global userid_industry_map
    global userid_age_map
    global userid_gender_map
        
    #global users_topics_probs
    
    global all_words_counts

    path = "blogs/"
    tokenizer_instance = Tokenizer()
    
    pattern = r'<post>(.*?)</post>'
    
    iterations = 0
    try:
        
        topic_words_file = pd.read_csv("wwbpFBtopics_condProb.csv")
        
        for filename in os.listdir(path):  #### For EACH USER
            
            iterations = iterations+1
            print iterations
            
            if iterations>500:
                break
                
            word_count_map = {}
            topic_prob_map = {}
            
            if filename.startswith("."):
                continue
                
            all_posts = []
            parts = filename.split(".")
            
            user_id = (int)(parts[0])
            gender = parts[1]
            age = (int)(parts[2])
            industry = parts[3]
            
            user_total_words_count = 0
            userid_age_map[user_id] = age
            
            if gender == "male":
                userid_gender_map[user_id] = 0
            else:
                userid_gender_map[user_id] = 1
                
            userid_industry_map[user_id] = industry

            
            if industry in industry_users_map:
                industry_users_map[industry] = industry_users_map[industry] + 1
            else:
                industry_users_map[industry] = 1
                
            
            full_filename = path+filename
            
            with open(full_filename, 'r') as myfile:
                text = myfile.read().replace('\n', '').replace('\r', '').replace('\t', '')
    
            all_posts = re.findall(pattern, text, re.DOTALL)
        
            total_number_of_blog_posts = total_number_of_blog_posts + len(all_posts)
            
            ### All the posts in this user's blog
            for blog_post in all_posts:
                words = tokenizer_instance.tokenize(blog_post)

                ### Total Number of Words This User uses
                user_total_words_count = user_total_words_count + len(words)
                
                for word in words:
                    if word in word_count_map:
                        word_count_map[word] = word_count_map[word] + 1
                    else:
                        word_count_map[word] = 1   
            
                    if word in all_words_counts:
                        all_words_counts[word] = all_words_counts[word] + 1
                    else:
                        all_words_counts[word] = 1      
                    
            #for word, user_wc in word_count_map.iteritems():
            for topic in range(2000):
                
                ### Initializing Prob of topic given user = 0
                prob_topic_given_user = 0.0

                word_row = topic_words_file[topic_words_file["category"]==topic]
                
                for index, row in word_row.iterrows():
                    word = row[0]
                    prob_topic_given_word = row[2]
                    
                    if word in word_count_map:
                        user_wc = word_count_map[word]
                        
                        prob_word_given_user = user_wc / user_total_words_count

                        cur = prob_topic_given_word * prob_word_given_user
                        prob_topic_given_user = prob_topic_given_user + cur
                    
                topic_prob_map[topic] = prob_topic_given_user
                        
                
            users_words_counts[user_id] = word_count_map
            users_topics_probs[user_id] = topic_prob_map
                
    except Exception as e:
        traceback.print_exc()
    
    print "1.a) posts: %d " % total_number_of_blog_posts
    print "1.b) users: %d " % len(userid_age_map)
    print "1.c) words: %d " % len(all_words_counts)
    print "1.d) "
    print industry_users_map

In [3]:
print userid_age_map

{}


In [4]:
if __name__ == '__main__':
    
    users_words_counts = {}
    users_topics_probs = {}

    read_all_blogs(users_words_counts,users_topics_probs)
    

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85


KeyboardInterrupt: 

### 2) Calculating first 3 users' probability of mentioning 3 given topics

In [ ]:
topicwordsfile = pd.read_csv("wwbpFBtopics_condProb.csv")

topic_map = {}

topic_map[463] = topicwordsfile[topicwordsfile["category"]==463]
topic_map[963] = topicwordsfile[topicwordsfile["category"]==963]
topic_map[981] = topicwordsfile[topicwordsfile["category"]==981]

#print topic_map[963]

lowest_user_ids = [5114,7596,8173]

print "2.a)"
for userid in lowest_user_ids:
    print "%d" %userid
    
    for topic in topic_map:
        print str(topic) + " : "
        
        prob_topic_given_user = users_topics_probs[topic]
        print prob_topic_given_user
        

### 3) Correlate each topic usage with user age, adjusting for gender

In [8]:
def multiply_matrices(Topics, Genders, Ages):
    Ones = []
    for i in range(len(Topics)):
        Ones.append(1)
        
    Right = np.matrix([Topics,Genders,Ones])
    Right = Right.T
    
    Left = np.matrix([Ages])
    Left = Left.T
    
    print Right
    
    Right_T = Right.T
    
    XT_X = Right_T * Right
    XT_X_I = XT_X.I
    
    XT_X_I_XT = XT_X_I * Right_T
    
    Betas = XT_X_I_XT * Left
    
    return Betas
    

In [9]:
# T is topic usage , A is User's Age , G is User's Gender
topic_words_file = pd.read_csv("wwbpFBtopics_condProb.csv")
print len(topic_words_file)

Ages = userid_age_map.values()
Genders = userid_gender_map.values()

Beta_topics = []
Beta_C = []

for topic in range(2000):
    print topic

    Topics = []
    topic_pd = topic_words_file[topic_words_file["category"]==topic]
    
    iterc =0
    for user_id,age in userid_age_map.iteritems():

        iterc = iterc + 1
        if iterc%1000 == 0:
            print iterc
            
        gender = userid_gender_map[user_id]
        user_word_counts_map = users_words_counts[user_id]

        user_all_words_count = 0
        for w ,c in user_word_counts_map.iteritems():
            user_all_words_count = user_all_words_count + c
    
        prob_topic_given_user = 0
        
        for index, row in topic_pd.iterrows():
            word = row[0]
            prob_topic_given_word = row[2]

            if word in user_word_counts_map:
                user_wc = user_word_counts_map[word]

                prob_word_given_user = user_wc / user_all_words_count

                cur = prob_topic_given_word * prob_word_given_user
                prob_topic_given_user = prob_topic_given_user + cur

        
        Topics.append(prob_topic_given_user)
    
    Betas = multiply_matrices(Topics,Genders,Ages)
    
    Beta_topic = Betas.item(0)
    Beta_c = Betas.item(2)
    
    Beta_C.append(Beta_c)
    Beta_topics.append(Beta_topic)
    
    print Beta_topics
    break


306244
0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
[[  2.99160083e-05   1.00000000e+00   1.00000000e+00]
 [  2.51238251e-05   0.00000000e+00   1.00000000e+00]
 [  5.48985766e-05   0.00000000e+00   1.00000000e+00]
 ..., 
 [  1.14318328e-05   0.00000000e+00   1.00000000e+00]
 [  1.54081162e-05   0.00000000e+00   1.00000000e+00]
 [  2.91616897e-05   1.00000000e+00   1.00000000e+00]]
[-846.926092720302]


In [2]:
import scipy.stats as ss
print ss.binom(100,0.1667).ppf(0.975)

24.0


In [25]:
normal = ss.norm(0,1)
print ss.t.sf(0.05, 4)

0.481259759221


In [2]:
import scipy.stats as ss
ss.t.sf(2.57,0.257)

0.28131261433417232

In [4]:
(((19.175**2 + 18.11**2))/(19.175**2/10 + 18.11**2/10)**2)*0.9

0.1293748975108234

In [6]:
ss.t.isf(0.95,0.129)

-10648037.704476822

In [8]:
import numpy as np
alum  = np.array([136, 73, 118, 122, 114, 103, 149, 118, 113, 105])

In [15]:
import math
math.sqrt(alum.var()*10/9)

20.21248239193901

In [39]:
X = [[-1,2],[ 4 ,12],[ 3,  5],[ 4  ,6],[-3,  9],[ 6  ,7]]
X = np.matrix(X)
std1 = np.std(X[:,0])
print std1
print (X[:,0] - np.mean(X[:,0]))
std2 = np.std(X[:,1])
print std2
print np.mean(X[:,1])

3.13138237134
[[-3.16666667]
 [ 1.83333333]
 [ 0.83333333]
 [ 1.83333333]
 [-5.16666667]
 [ 3.83333333]]
3.13138237134
6.83333333333


In [33]:
X[:,1] = (X[:,1] - np.mean(X[:,1]))/np.std(X[:,1])

In [35]:
X[:,0] = (X[:,0] - np.mean(X[:,0]))/np.std(X[:,0])

In [36]:
X

matrix([[-1, -1],
        [ 0,  1],
        [ 0,  0],
        [ 0,  0],
        [-1,  0],
        [ 1,  0]])